In [5]:
! pip install pytube
! pip install speechrecognition
! pip install pyttsx3
! pip install string
! pip install re 
! pip install pytube
! pip install SpeechRecognition
! pip install googletrans==4.0.0-rc1
! pip install gTTS

ERROR: Could not find a version that satisfies the requirement string (from versions: none)
ERROR: No matching distribution found for string
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [2]:
from pytube import YouTube
import speech_recognition as sr
from googletrans import Translator
import pyttsx3

In [3]:
# Step 1: Extract Video Information
def extract_video_info(youtube_link):
    yt = YouTube(youtube_link)
    video_title = yt.title
    video_captions = yt.captions.get_by_language_code('en')
    return video_title, video_captions

# Step 2: Convert Audio to Text
def convert_audio_to_text(captions):
    if captions is None:
        print("Captions not available for this video.")
        return None  # Return None if captions are not available
    else:
        audio_transcript = captions.generate_srt_captions()
        return audio_transcript

# Step 3: Text Translation
def translate_text(text, target_language):
    if text is None:
        return None  # Return None if there's no text to translate
    translator = Translator()
    translated_text = translator.translate(text, dest=target_language)
    return translated_text.text

# Step 4: Text-to-Speech Conversion
def convert_text_to_speech(translated_text, target_language):
    if translated_text is None:
        print("No translated text available to convert to speech.")
        return
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)    # You can adjust the speaking rate
    engine.setProperty('voice', f'{target_language}')  # Adjust voice if available
    engine.say(translated_text)
    engine.runAndWait()

if __name__ == "__main__":
    youtube_link = input("Enter the YouTube video link: ")
    target_language = input("Enter the target language: ")

    video_title, captions = extract_video_info(youtube_link)
    audio_transcript = convert_audio_to_text(captions)

    translated_text = translate_text(audio_transcript, target_language)
    convert_text_to_speech(translated_text, target_language)


Captions not available for this video.
No translated text available to convert to speech.


C:\Users\HP\AppData\Local\Temp\ipykernel_16072\568595975.py:5: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  video_captions = yt.captions.get_by_language_code('en')


In [4]:
import speech_recognition as sr
from googletrans import Translator
import pyttsx3
from pytube import YouTube

# Step 1: Extract Audio from YouTube Video
def extract_audio(youtube_link, file_format='mp3', output_path=None):
    yt = YouTube(youtube_link)
    audio_stream = yt.streams.filter(only_audio=True, file_extension=file_format).first()
    if audio_stream:
        if output_path:
            audio_file = audio_stream.download(output_path=output_path, filename="video_audio")
        else:
            audio_file = audio_stream.download(filename="video_audio")
        return audio_file
    else:
        print("No audio stream found.")
        return None


# Step 2: Speech Recognition
def recognize_speech(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
    try:
        transcript = recognizer.recognize_google(audio_data)
        return transcript
    except sr.UnknownValueError:
        print("Speech recognition could not understand audio.")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service: {e}")
        return None

# Step 3: Text Translation
def translate_text(text, target_language):
    translator = Translator()
    translated_text = translator.translate(text, dest=target_language)
    return translated_text.text

# Step 4: Text-to-Speech Conversion
def convert_text_to_speech(translated_text, target_language):
    engine = pyttsx3.init()
    engine.setProperty('rate', 150)    # You can adjust the speaking rate
    engine.setProperty('voice', f'{target_language}')  # Adjust voice if available
    engine.say(translated_text)
    engine.runAndWait()

if __name__ == "__main__":
    youtube_link = input("Enter the YouTube video link: ")
    target_language = input("Enter the target language: ")
    output_path = input("Enter the output directory path: ")

    # Step 1: Extract Audio from YouTube Video
    audio_file = extract_audio(youtube_link, output_path)

    if audio_file:
        # Step 2: Speech Recognition
        transcript = recognize_speech(audio_file)

        if transcript:
            # Step 3: Text Translation
            translated_text = translate_text(transcript, target_language)
            # Step 4: Text-to-Speech Conversion
            convert_text_to_speech(translated_text, target_language)
        else:
            print("Speech recognition could not understand audio.")
    else:
        print("No audio stream found.")


ValueError: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format

In [ ]:
# from pytube import YouTube
# import os

# # Step 1: Extract Audio from YouTube Video
# def extract_audio(youtube_link, output_path):
#     yt = YouTube(youtube_link)
#     audio_streams = yt.streams.filter(only_audio=True)
    
#     # Filter streams by file extension
#     mp3_stream = audio_streams.filter(file_extension='mp4').first()
#     wav_stream = audio_streams.filter(file_extension='webm').first()

#     if mp3_stream:
#         mp3_file = mp3_stream.download(output_path=output_path, filename='audio_mp3')
#         print(f"MP3 Audio downloaded to: {mp3_file}")
#     else:
#         print("No MP3 audio stream found.")

#     if wav_stream:
#         wav_file = wav_stream.download(output_path=output_path, filename='audio_wav')
#         print(f"WAV Audio downloaded to: {wav_file}")
#     else:
#         print("No WAV audio stream found.")

# if __name__ == "__main__":
#     youtube_link = input("Enter the YouTube video link: ")
#     output_patneh = input("Enter the output directory path: ")

#     if not os.path.exists(output_path):
#         os.makedirs(output_path)

#     extract_audio(youtube_link, output_path)

MP3 Audio downloaded to: C:\Users\HP\Desktop\final-sts-flask\audio_mp3
WAV Audio downloaded to: C:\Users\HP\Desktop\final-sts-flask\audio_wav


### Below is the working code which extracts audio from youtube video

In [6]:
from pytube import YouTube
import os
import string
import re

def download_audio(url):
    yt = YouTube(url)
    video = yt.streams.filter(only_audio=True).first()

    # Sanitize the video title to remove invalid characters
    valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
    video_title = ''.join(c for c in yt.title if c in valid_chars)
    video_title = re.sub(r'\s+', '_', video_title)  # Replace spaces with underscores

    # Download audio as MP3
    print("Downloading MP3...")
    output_mp3 = video.download(filename=f"{video_title}.mp3")
    base, ext = os.path.splitext(output_mp3)
    new_file_mp3 = base + '.mp3'
    os.rename(output_mp3, new_file_mp3)

    # Download audio as WAV
    print("Downloading WAV...")
    output_wav = video.download(filename=f"{video_title}.wav")
    base, ext = os.path.splitext(output_wav)
    new_file_wav = base + '.wav'
    os.rename(output_wav, new_file_wav)

    print(f"Downloaded: {new_file_mp3} and {new_file_wav}")  

if __name__ == "__main__":
    url = input("Enter the YouTube video URL: ")
    download_audio(url)

Downloaded: c:\Users\HP\Desktop\final-sts-flask\Is_Sade_Sati_The_Worst_Time_Of_Your_Life_Gurudev.mp3 and c:\Users\HP\Desktop\final-sts-flask\Is_Sade_Sati_The_Worst_Time_Of_Your_Life_Gurudev.wav


In [ ]:
# import os
# import string
# import re
# from pytube import YouTube
# import speech_recognition as sr
# from googletrans import Translator
# from gtts import gTTS

# def download_audio(url):
#     yt = YouTube(url)
#     video = yt.streams.filter(only_audio=True).first()

#     # Sanitize the video title to remove invalid characters
#     valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
#     video_title = ''.join(c for c in yt.title if c in valid_chars)
#     video_title = re.sub(r'\s+', '_', video_title)  # Replace spaces with underscores

#     # Download audio as MP3
#     print("Downloading MP3...")
#     output_mp3 = video.download(filename=f"{video_title}.mp3")
#     base, ext = os.path.splitext(output_mp3)
#     new_file_mp3 = base + '.mp3'
#     os.rename(output_mp3, new_file_mp3)

#     return new_file_mp3

# def transcribe_audio(audio_file, language='en-US'):
#     r = sr.Recognizer()

#     with sr.AudioFile(audio_file) as source:
#         audio_data = r.record(source)

#     try:
#         text = r.recognize_google(audio_data, language=language)
#         return text
#     except sr.UnknownValueError:
#         print("Unable to recognize speech.")
#     except sr.RequestError as e:
#         print(f"Error; {e}")

# def translate_text(text, target_language='hi'):
#     translator = Translator()
#     translated_text = translator.translate(text, dest=target_language).text
#     return translated_text

# def text_to_speech(text, language='hi', output_file='translated_audio.mp3'):
#     tts = gTTS(text=text, lang=language)
#     tts.save(output_file)
#     print(f"Translated audio saved as {output_file}")

# if __name__ == "__main__":
#     url = input("Enter the YouTube video URL: ")
#     audio_file = download_audio(url)
#     original_text = transcribe_audio(audio_file)
#     print(f"Original Text: {original_text}")

#     translated_text = translate_text(original_text, target_language='hi')
#     print(f"Translated Text: {translated_text}")

#     text_to_speech(translated_text, language='hi', output_file='hindi_audio.mp3')

RegexMatchError: regex_search: could not find match for (?:v=|\/)([0-9A-Za-z_-]{11}).*

In [ ]:
# import os
# import string
# import re
# from pytube import YouTube
# import speech_recognition as sr
# from googletrans import Translator
# from gtts import gTTS

# def download_audio(url):
#     yt = YouTube(url)
#     video = yt.streams.filter(only_audio=True).first()

#     # Sanitize the video title to remove invalid characters
#     valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
#     video_title = ''.join(c for c in yt.title if c in valid_chars)
#     video_title = re.sub(r'\s+', '_', video_title)  # Replace spaces with underscores

#     # Download audio as MP3
#     print("Downloading MP3...")
#     output_mp3 = video.download(filename=f"{video_title}.mp3")
#     base, ext = os.path.splitext(output_mp3)
#     new_file_mp3 = base + '.mp3'
#     os.rename(output_mp3, new_file_mp3)

#     return new_file_mp3

# def transcribe_audio(audio_file, language='en-US'):
#     r = sr.Recognizer()

#     with sr.AudioFile(audio_file) as source:
#         audio_data = r.record(source)

#     try:
#         text = r.recognize_google(audio_data, language=language)
#         return text
#     except sr.UnknownValueError:
#         print("Unable to recognize speech.")
#     except sr.RequestError as e:
#         print(f"Error; {e}")

# def translate_audio(audio_file, target_language):
#     original_text = transcribe_audio(audio_file)
#     print(f"Original Text: {original_text}")

#     translator = Translator()
#     translated_text = translator.translate(original_text, dest=target_language).text
#     print(f"Translated Text: {translated_text}")

#     return translated_text

# def text_to_speech(text, language, output_file):
#     tts = gTTS(text=text, lang=language)
#     tts.save(output_file)
#     print(f"Translated audio saved as {output_file}")

# if __name__ == "__main__":
#     url = input("Enter the YouTube video URL: ")
#     audio_file = download_audio(url)

#     target_language = input("Enter the target language code (e.g., 'hi' for Hindi): ")

#     translated_text = translate_audio(audio_file, target_language)

#     output_file = f"translated_audio_{target_language}.mp3"
#     text_to_speech(translated_text, target_language, output_file)

ValueError: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format